In [66]:
import os

from IPython.display import display, Markdown
import pandas as pd
from fuzzywuzzy import fuzz
import spacy
from pprint import pprint

from src.scraping import extract_cnbc_article_info
from src.nlp import nlp_analysis
from src.urls import article_urls

In [48]:
article_urls

['https://www.cnbc.com/2019/03/15/spielberg-and-netflixs-sarandos-meet-sparking-hope-for-oscars-truce.html',
 'https://www.cnbc.com/2019/03/15/tesla-shares-fall-5-percent-after-company-unveils-model-y.html',
 'https://www.cnbc.com/2019/03/15/arizona-will-go-after-big-tech-if-necessary-says-attorney-general.html',
 'https://www.cnbc.com/2019/03/15/facebook-stocks-falls-after-cox-exit-outage-threats-from-washington.html',
 'https://www.cnbc.com/2019/02/24/kraft-heinz-reviews-options-for-maxwell-house-coffee-including-sale.html',
 'https://www.cnbc.com/2019/03/08/jeff-bezos-to-end-secrecy-over-amazons-role-in-carbon-emissions.html',
 'https://www.cnbc.com/2019/03/15/dieselgate-sec-says-volkswagen-perpetrated-fraud-lied-to-investors.html']

### Scrape CNBC Website for Articles

In [49]:
articles = []
for article_id, article_url in enumerate(article_urls):
    article_text = extract_cnbc_article_info(article_url)
    article_nlp = nlp_analysis(article_text)

    articles.append({
        'article_id': article_id,
        **article_text, 
        **article_nlp, 
    })


In [67]:
pprint(articles)

[{'article': '\n'
             'Steven Spielberg, a film industry titan with 58 directorial '
             'credits to his name, has long been vocal about the differences '
             'between theatrical and streaming releases.\n'
             '\n'
             'As the governor of the Academy of Motion Picture Arts and '
             'Sciences directors branch he has said that he would like films '
             'to have a longer theatrical run in order to qualify for an '
             'Academy Award. Those comments have led some to believe that '
             'Spielberg was ready to wage war on platforms like Netflix, '
             'especially after the streaming service\'s film "Roma" took home '
             'three Oscars last month.\n'
             "However, Spielberg reportedly met with Netflix's Chief Content "
             'Officer Ted Sarandos earlier this week, a sign that, perhaps, '
             'these rumors of discord are just that, rumors.\n'
             '\n'
         

### Correlate with IA Holdings

In [56]:
investment_advisor_holdings = pd.read_csv(
    os.path.join('data', 'internal', 'investment_advisor_holdings.csv'))

In [57]:
investment_advisor_holdings

,investment_advisor,security_name,security_id,market_value
0,Greg Warren,Alphabet Inc,GOOGL,1429000
1,Greg Warren,"Netflix, Inc.",NFLX,2353000
2,Greg Warren,Enbridge Inc,ENB,154000
3,Greg Warren,Royal Bank of Canada,RY,4649000
4,Greg Warren,Suncor Energy Inc.,SU,1999000
5,Greg Warren,Volkswagen AG,VOW3,991000
6,Greg Warren,"Amazon.com, Inc.",AMZN,743000
7,Greg Warren,Toronto-Dominion Bank,TD,3703000
8,Greg Warren,Kraft Heinz Co,KHC,4135000
9,Michelle Burns,"Facebook, Inc.",FB,928000


In [63]:
IA_articles = []

investment_advisors = investment_advisor_holdings['investment_advisor'].unique()
for ia in investment_advisors:
    articles_matched = []
    
    ia_holdings = investment_advisor_holdings[
        investment_advisor_holdings['investment_advisor']==ia]

    for article in articles:
        article_match = False
        article_id = article['article_id']
        organizations = [orgs[0] for orgs in article['organizations']]
        for org in organizations:
            for security_name in ia_holdings['security_name'].unique():
                org_sim = fuzz.token_set_ratio(org, security_name)
                if org_sim > 80:
                    articles_matched.append(article_id)
                    article_match = True
                    break
            if article_match:
                break
    IA_articles.append({
        'investment_advisor': ia,
        'matched_articles': articles_matched
    })

In [64]:
IA_articles

[{'investment_advisor': 'Greg Warren', 'matched_articles': [0, 3, 4, 5, 6]},
 {'investment_advisor': 'Michelle Burns', 'matched_articles': [1, 2, 3, 6]}]

In [65]:
for ia_article in IA_articles:
    display(Markdown(f"# Daily recommended articles for {ia_article['investment_advisor']}"))
    matched_articles = ia_article['matched_articles']
    for article in articles:
        if article['article_id'] in matched_articles:
            display(Markdown(f"### { article['title']}"))
            display(Markdown(f"Summary: {' '.join(article['pagerank_summary'])}"))
            display(Markdown(f"Read more: {article['url']}"))
  

# Daily recommended articles for Greg Warren

### Steven Spielberg reportedly met with Netflix's Ted Sarandos, sparking hopes for Academy Awards' truce

Summary: Earlier this week, media mogul Jeffrey Katzenberg said that Spielberg had no plan to campaign against Netflix, saying at the South By Southwest conference in Texas that "he is not going to the academy in April with some sort of plan," according to a report by the Hollywood Reporter. Those comments have led some to believe that Spielberg was ready to wage war on platforms like Netflix, especially after the streaming service's film "Roma" took home three Oscars last month. As the governor of the Academy of Motion Picture Arts and Sciences directors branch he has said that he would like films to have a longer theatrical run in order to qualify for an Academy Award.

Read more: https://www.cnbc.com/2019/03/15/spielberg-and-netflixs-sarandos-meet-sparking-hope-for-oscars-truce.html

### Facebook stock falls after the executive in charge of all its products steps down

...

Summary:  Facebook stock fell Friday, the day after Chief Product Officer Chris Cox announced he was leaving the company in light of its recent pivot to privacy. Meanwhile, federal prosecutors are probing Facebook's data-sharing partnerships with device makers, according to a report by the New York Times published Wednesday night. The company was alerted to the post by New Zealand police and said it has been taking down re-postings of the video and messages praising the shooter as it becomes aware of them.

Read more: https://www.cnbc.com/2019/03/15/facebook-stocks-falls-after-cox-exit-outage-threats-from-washington.html

### SEC charges Volkswagen and its former CEO with defrauding investors

Summary: VW had said in its annual report that the SEC could take enforcement action against the company over the German automaker's involvement in the emissions scandal. The SEC does not charge that any person involved in the bond issuance knew that Volkswagen diesel vehicles did not comply with U.S. emissions rules when these securities were sold, but simply repeats unproven claims about Volkswagen AG's former CEO, who played no part in the sales. The complaint claims Volkswagen made false and misleading statements to investors and underwriters about vehicle quality, environmental compliance and the company's financial standing, which gave Volkswagen a financial benefit when it issued securities at more attractive rates for the company.

Read more: https://www.cnbc.com/2019/03/15/dieselgate-sec-says-volkswagen-perpetrated-fraud-lied-to-investors.html